# Train a bigger pix2pixHD on binary masks 1024 aggresive augmentation

In [ ]:
!cd ../.. ; python src/models/Pix2pixHD/train.py --continue_train --ngf 80 --ndf 40 --nef 24 --niter 80 --niter_decay 80 --checkpoints_dir temp/checkpoint/Pix2pixHD/ --name xBD_full_conditional_binary --dataroot xBD_full --dataset_mode physics_aligned_bin --no_instance --label_nc 0 --input_nc 4 --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7

In [ ]:
#cp pretrained model into finetuning directory
#Make a copy of the fully trained model to be finetuned on xBD_floods
!cd ../../temp/checkpoint/Pix2pixHD/; mkdir conditional_binary_finetuned_augmented ;  cp xBD_full_conditional_binary/latest* conditional_binary_finetuned_augmented/ ; cp xBD_full_conditional_binary/iter* conditional_binary_finetuned_augmented/

In [ ]:
#Finetuning step
!cd ../.. ; python src/models/Pix2pixHD/train.py --continue_train --ngf 80 --ndf 40 --nef 24 --niter 110 --niter_decay 110 --checkpoints_dir temp/checkpoint/Pix2pixHD/ --name conditional_binary_finetuned_augmented --dataroot xBD_flood_augmented --dataset_mode physics_aligned_bin --no_instance --label_nc 0 --input_nc 4 --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7

In [41]:
import time
time.sleep(6000)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-41-db8f6f367f10>", line 2, in <module>
    time.sleep(6000)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1148, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/ultratb.py", line 316, in wrapp

KeyboardInterrupt: 

In [ ]:
#Finetuning step with the only non-augmented samples two epochs low Lr
!cd ../.. ; python src/models/Pix2pixHD/train.py --continue_train --ngf 80 --ndf 40 --nef 24 --niter 120 --niter_decay 120 --checkpoints_dir temp/checkpoint/Pix2pixHD/ --name conditional_binary_finetuned_augmented --dataroot xBD_flood_augmented_noelastic --dataset_mode physics_aligned_bin --no_instance --label_nc 0 --input_nc 4 --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7

In [ ]:
#Generate predictions over the test set. 
!cd ../.. ; python src/models/Pix2pixHD/test.py --checkpoints_dir temp/checkpoint/Pix2pixHD/  --ngf 80 --nef 24 --phase test --how_many 2000 --name conditional_binary_finetuned_augmented --dataroot xBD --dataset_mode physics_aligned_bin --no_instance --label_nc 0 --batchSize 8 --input_nc 4 --gpu_ids 0,1,2,3,4,5,6,7 --results_dir temp/Pix2pixHD/


### We move things around at the moment in ./Temp/
The model output is placed into ./temp/ .We use this directory to work, then we carefully selects what goes into ./results/.
./temp/ does not get uploaded into our bucket, only ./results/ do!

In [ ]:
# Move synthesized results to a folder to be used by the segmentation network
!cd ../../temp/Pix2pixHD/conditional_binary_finetuned_augmented/test_latest/ ; mkdir to_segment; cp -R images/*synthesized_image.jpg to_segment

In [ ]:
#Create Segmentation Masks
#Generate masks for all of the data generated by pix2pixHD_baseline
! cd ../.. ; python src/models/Pix2pix-CycleGAN/test.py --phase to_segment --dataroot temp/Pix2pixHD/conditional_binary_finetuned_augmented/test_latest/ --direction AtoB --results_dir ./temp/Pix2pixHD/conditional_binary_finetuned_augmented_masks --dataset_mode single --model pix2pix --checkpoints_dir ./pretrained/Pix2pix-CycleGAN/flood_segmentation/scratch_1024_plus/ --name . --num_test 775 --no_flip --gpu_ids 0 --max_dataset_size 2000 --batch_size 1  --load_size 1024 --crop_size 1024


### Now we copy our generated images into the right ./Results/. directory

In [ ]:
! cd ../.. ; mkdir results/Pix2pixHD/conditional_binary_finetuned_augmented/ ; cp -R temp/Pix2pixHD/conditional_binary_finetuned_augmented/test_latest/images/* results/Pix2pixHD/conditional_binary_finetuned_augmented/ ; cp -R temp/Pix2pixHD/conditional_binary_finetuned_augmented_masks/to_segment_latest/images/*image_fake_B.png results/Pix2pixHD/conditional_binary_finetuned_augmented/